In [13]:
import sys, threading
import numpy as np
import random
import time
import matplotlib.pyplot as plt
sys.setrecursionlimit(10 ** 7)
threading.stack_size(2 ** 27)
random.seed(time.time())

TODOS
- Revise methods that are completely "plagiarised"
- Need a method to distrubute Keys
- Need to divide messages to blocks according to key length
- Implement Essam's revision of the brute force attack
- Implement chosen ciphertext attack

DONE
- Copied from number theory: 
  - GCD - Extended Euclid - Modular Exponentiation - Modular Inverse - Convert to Int - Convert to String - Encrypt - Decrypt
- Generate random numbers of n binary bits
- Primality Check
- The two above points are responsible for public keys
- Generate private keys from public keys


In [14]:
def GCD(a, b):
    if b == 0:
        return a
    return GCD(b, a % b)

def extendedEuclid(a, b):
    if b == 0:
        return (1, 0)
    (x, y) = extendedEuclid(b, a % b)
    k = a // b
    return (y, x - k * y)

def modularExponentiate(a, n, mod):
    if n == 0:
        return 1 % mod
    elif n == 1:
        return a % mod
    # else:
    #     b = modularExponentiate(a, n // 2, mod)
    #     b = b * b % mod
    #     if n % 2 == 0:
    #         return b
    #     else:
    #         return b * a % mod

    #revised implementation from the reference
    f = 1
    binaryB = bin(n)[2:]
    for i in range(len(binaryB)):
        f = int(f*f) % mod
        if binaryB[i] == '1':
            f = int(f * a) % mod
    return f

# def revisedModExpo(a,n,mod):
#     f = 1
#     binaryB = bin(n)[2:]
#     for i in range(len(binaryB)):
#         f = (f*f) % mod
#         if binaryB[i] == '1':
#             f = (f * a) % mod
#     return f

    

def modularInverse(a, n):
    (b, x) = extendedEuclid(a, n)
    if b < 0:
        b = (b % n + n) % n  # we don't want -ve integers
    return b

#TODO: change the implementation
#look up ways to change strings to ints for encryption
def ConvertToInt(message_str):  
    res = 0
    for i in range(len(message_str)):
        res = res * 256 + ord(message_str[i])
    return res

#TODO: change the implementation
#look up ways to change strings to ints for encryption
def ConvertToStr(n):
    res = ""
    while n > 0:
        res += chr(n % 256)
        n //= 256
    return res[::-1]

def Encrypt(m, e, n):
    m = ConvertToInt(m)
    c = modularExponentiate(m,e,n)
    return c

def getPrivateKey(e,p,q):
    phi_n = (p - 1) * (q - 1)
    d = modularInverse(e, phi_n)
    return d

def Decrypt(c, d, p, q):
    m = modularExponentiate(c, d, p * q)
    m = ConvertToStr(m)
    return m

In [15]:
#simple test
p = 1000000007
q = 1000000009
exponent = 23917
modulo = p * q
ciphertext = Encrypt("attack", exponent, modulo)
d = getPrivateKey(exponent,p,q)
message = Decrypt(ciphertext, d, p, q)
#print(message)

x = bin(2)[2:] #bin return 0bxx, sliced from position 2to get rid of 0b
# print(x[1])

# a = modularExponentiate(55,97,1234)
# b = revisedModExpo(55,97,1234)
# print(a,b)


In [16]:
"""
Parameter generation
on sign up generate large numbers of preferably 512 bits to be used as p and q
then generate random e such that gcd(e,phi(n)) = 1
e can be generated using nBitRandom then checking using GCD that the result is 1
d is the modular inverse of e mod(phi(n))
"""

#generating p and q
#https://www.geeksforgeeks.org/how-to-generate-large-prime-numbers-for-rsa-algorithm/
def nBitRandom(n):
    # Returns a random number of n bits
    if n == 1 : return random.randrange(0,1)
    elif n == 2 : return random.randrange(0,3) 
    else: return np.int64(random.randrange(2**(n-1)+1, 2**n-1))

def fermatPrimalityTest(p):
    """
    a:random integer
    p:the number to test if prime or not
    """
    if p <= 1: return False
    for _ in range(1,102):
        a=np.random.randint(1,p,dtype=np.int64)
        aPowP = modularExponentiate(a,p,p)
        if (aPowP - a) % p != 0: return False
    return True



In [17]:
"""
Brute force attack
you need to factorize large n into 2 prime factors
factorization of n is unique, once you find 2 primes ur done

"""
# brute force attack

#responsible for factorization
def bruteForceAttack(n):
    #remember the properties of factorization, we only need to try up till square root of n
    for i in range(2,np.sqrt(n)):
        if n % i == 0:
            return i

In [18]:
# Generate prime numbers less than N
def generatePrimeNumbers(N):
    # Returns a list of prime numbers less than N
    primeNumbers = []
    for i in range(2, N):
        if fermatPrimalityTest(i):  # Fermat's primality test
            primeNumbers.append(i)
    return primeNumbers

def generatePrime(n):
    if n == 1: return -1
    number = 1 
    while not fermatPrimalityTest(number):
        number = np.int64(nBitRandom(n))
    return number


In [19]:
def testBruteForceTime(N):
    time_taken = []
    n_values =[]
    # Test the time it takes to brute force attack
    primeNumbers = generatePrimeNumbers(N) # Generate prime numbers less than N
    for i in range(len(primeNumbers)):
        for j in range(i+1, len(primeNumbers)):
            n_val =primeNumbers[i]*primeNumbers[j] # n = p*q
            n_values.append(n_val)
            start_time = time.time()
            bruteForceAttack(n_val) # Brute force attack
            end_time = time.time()
            time_taken.append(end_time - start_time) # Time taken to brute force attack
    return time_taken , n_values

In [20]:
# time_vals , n_vals = testBruteForceTime(10000)

In [21]:
# plt.plot(n_vals, time_vals)
# plt.xlabel('N')
# plt.ylabel('Time taken')
# plt.title('Brute Force Attack')
# plt.show()

NameError: name 'n_vals' is not defined

In [22]:
#revised brute force attack
def bruteForceTime():
    n = 64
    p = 1
    q = 1
    nArray = []
    for i in range(2,int(n/2)+1):
        # for _ in range(2):
            # while (not fermatPrimalityTest(p)):
            #     p = nBitRandom(i)
            # while (not fermatPrimalityTest(q)):
            #     q = nBitRandom(i)
        p = generatePrime(i)
        q = generatePrime(i)
        nArray.append(np.int64(p*q))

    time_taken = []
    start_time = time.time()
    bruteForceAttack(n_val for n_val in nArray) # Brute force attack
    end_time = time.time()
    time_taken.append(end_time - start_time) # Time taken to brute force attack
    return nArray, time_taken

# bruteForceTime()
    

C:\Users\user\AppData\Local\Temp/ipykernel_12512/2630633176.py:30: RuntimeWarning: overflow encountered in longlong_scalars
  f = int(f*f) % mod
C:\Users\user\AppData\Local\Temp/ipykernel_12512/2630633176.py:32: RuntimeWarning: overflow encountered in longlong_scalars
  f = int(f * a) % mod


[4,
 25,
 143,
 437,
 3599,
 8137,
 23213,
 227429,
 404207,
 2044397,
 7846477,
 25245461,
 190291831,
 566914669,
 1802325359,
 10917045791,
 43942943713,
 125595163219,
 658865098637,
 1951989017029,
 10251891458593,
 36947331257897,
 226466212318237,
 1004658309382667,
 1429523504793959,
 9129647095076077,
 26317545813666629,
 178662051998561323,
 982235024288361353,
 2609396824420496969,
 5440146277439107679]